In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime

from pandas import DataFrame, Series
%matplotlib inline

In [2]:
! ls ../work_data/

Instructions.odt  sample_submission.csv  store.csv  test.csv  train.csv


In [3]:
testcsvPath = '../work_data/test.csv'
!cat {testcsvPath} | head -n 5

"Id","Store","DayOfWeek","Date","Open","Promo","StateHoliday","SchoolHoliday"
1,1,4,2015-09-17,1,1,"0","0"
2,3,4,2015-09-17,1,1,"0","0"
3,7,4,2015-09-17,1,1,"0","0"
4,8,4,2015-09-17,1,1,"0","0"
cat: write error: Broken pipe


In [4]:
! csvstat {testcsvPath}

  1. Id
	<type 'int'>
	Nulls: False
	Min: 1
	Max: 41088
	Sum: 844132416
	Mean: 20544.5
	Median: 20544.5
	Standard Deviation: 11861.0839267
	Unique values: 41088
  2. Store
	<type 'int'>
	Nulls: False
	Min: 1
	Max: 1115
	Sum: 22840800
	Mean: 555.89953271
	Median: 553.5
	Standard Deviation: 320.270598497
	Unique values: 856
	5 most frequent values:
		344:	48
		345:	48
		346:	48
		347:	48
		340:	48
  3. DayOfWeek
	<type 'int'>
	Nulls: False
	Min: 1
	Max: 7
	Sum: 163496
	Mean: 3.97916666667
	Median: 4.0
	Standard Deviation: 2.01545676516
	Unique values: 7
	5 most frequent values:
		1:	5992
		3:	5992
		2:	5992
		4:	5992
		7:	5992
  4. Date
	<type 'datetime.date'>
	Nulls: False
	Min: 2015-08-01
	Max: 2015-09-17
	Unique values: 48
	5 most frequent values:
		2015-08-31:	856
		2015-08-28:	856
		2015-08-29:	856
		2015-08-08:	856
		2015-08-09:	856
  5. Open
	<type 'int'>
	Nulls: True
	Values: 0, 1
  6. Promo
	<type 'int'>
	Nulls: False
	Values: 0, 1
  7. StateHoliday
	<type 'unicode'>
	Nulls: Fal

# Files
<p>
train.csv - historical data including Sales<br/>
test.csv - historical data excluding Sales<br/>
sample_submission.csv - a sample submission file in the correct format<br/>
store.csv - supplemental information about the stores<br/>
</p>
#Data fields

<p>Most of the fields are self-explanatory. The following are descriptions for those that aren't.<br/>
Id - an Id that represents a (Store, Date) duple within the test set<br/>
Store - a unique Id for each store<br/>
Sales - the turnover for any given day (this is what you are predicting)<br/>
Customers - the number of customers on a given day<br/>
Open - an indicator for whether the store was open: 0 = closed, 1 = open<br/>
StateHoliday - indicates a state holiday. Normally all stores, with few exceptions, are closed on state holidays. Note that all schools are closed on public holidays and weekends. a = public holiday, b = Easter holiday, c = Christmas, 0 = None<br/>
SchoolHoliday - indicates if the (Store, Date) was affected by the closure of public schools<br/>
StoreType - differentiates between 4 different store models: a, b, c, d<br/>
Assortment - describes an assortment level: a = basic, b = extra, c = extended<br/>
CompetitionDistance - distance in meters to the nearest competitor store<br/>
CompetitionOpenSince[Month/Year] - gives the approximate year and month of the time the nearest competitor was opened<br/>
Promo - indicates whether a store is running a promo on that day<br/>
Promo2 - Promo2 is a continuing and consecutive promotion for some stores: 0 = store is not participating, 1 = store is participating<br/>
Promo2Since[Year/Week] - describes the year and calendar week when the store started participating in Promo2<br/>
PromoInterval - describes the consecutive intervals Promo2 is started, naming the months the promotion is started anew.<br/> E.g. "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store
</p>

In [5]:
!ls ../work_data/train.csv

../work_data/train.csv


In [6]:
trainPath = "../work_data/train.csv"

In [7]:
! cat {trainPath}|head -n 5

"Store","DayOfWeek","Date","Sales","Customers","Open","Promo","StateHoliday","SchoolHoliday"
1,5,2015-07-31,5263,555,1,1,"0","1"
2,5,2015-07-31,6064,625,1,1,"0","1"
3,5,2015-07-31,8314,821,1,1,"0","1"
4,5,2015-07-31,13995,1498,1,1,"0","1"
cat: write error: Broken pipe


In [9]:
dfTraincsv = pd.read_csv(trainPath)

In [10]:
dfTraincsv.describe()

,Store,DayOfWeek,Sales,Customers,Open,Promo,SchoolHoliday
count,1017209.000000,1017209.000000,1017209.000000,1017209.000000,1017209.000000,1017209.000000,1017209.000000
mean,558.429727,3.998341,5773.818972,633.145946,0.830107,0.381515,0.178647
std,321.908651,1.997391,3849.926175,464.411734,0.375539,0.485759,0.383056
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,280.000000,2.000000,3727.000000,405.000000,1.000000,0.000000,0.000000
50%,558.000000,4.000000,5744.000000,609.000000,1.000000,0.000000,0.000000
75%,838.000000,6.000000,7856.000000,837.000000,1.000000,1.000000,0.000000
max,1115.000000,7.000000,41551.000000,7388.000000,1.000000,1.000000,1.000000


In [11]:
dfTraincsv.corr()

,Store,DayOfWeek,Sales,Customers,Open,Promo,SchoolHoliday
Store,1.000000,-0.000008,0.005126,0.024325,-0.000047,0.000058,0.000641
DayOfWeek,-0.000008,1.000000,-0.462125,-0.386445,-0.528963,-0.392925,-0.205388
Sales,0.005126,-0.462125,1.000000,0.894711,0.678472,0.452345,0.085124
Customers,0.024325,-0.386445,0.894711,1.000000,0.616768,0.316169,0.071568
Open,-0.000047,-0.528963,0.678472,0.616768,1.000000,0.295042,0.086171
Promo,0.000058,-0.392925,0.452345,0.316169,0.295042,1.000000,0.067483
SchoolHoliday,0.000641,-0.205388,0.085124,0.071568,0.086171,0.067483,1.000000


In [30]:
group_by_DayOfWeek = dfTraincsv.groupby("DayOfWeek")
dfTraincsvByDayOfWeek = group_by_DayOfWeek.sum().drop("Store",1)
dfTraincsvByDayOfWeek

,Sales,Customers,Open,Promo,SchoolHoliday
DayOfWeek,,,,,
1,1130203012,117675012,137560,77760,34060
2,1020411930,110848063,143961,77580,36595
3,954962863,105117642,141936,77580,34636
4,911177709,101732938,134644,77580,34747
5,980555941,108384820,138640,77580,36235
6,846317735,95103854,144058,0,2724
7,29551433,5179426,3593,0,2724


In [34]:
dfTraincsvByDayOfWeek["SalesPerCustomer"] = dfTraincsvByDayOfWeek["Sales"]/dfTraincsvByDayOfWeek["Customers"]
dfTraincsvByDayOfWeek.corr()

,Sales,Customers,Open,Promo,SchoolHoliday,SalesPerCustomer
Sales,1.000000,0.997778,0.964575,0.745242,0.741658,0.996020
Customers,0.997778,1.000000,0.978076,0.734644,0.732648,0.997122
Open,0.964575,0.978076,1.000000,0.618096,0.618845,0.980040
Promo,0.745242,0.734644,0.618096,1.000000,0.998347,0.696359
SchoolHoliday,0.741658,0.732648,0.618845,0.998347,1.000000,0.691564
SalesPerCustomer,0.996020,0.997122,0.980040,0.696359,0.691564,1.000000


In [35]:
dfTraincsvByDayOfWeek

,Sales,Customers,Open,Promo,SchoolHoliday,SalesPerCustomer
DayOfWeek,,,,,,
1,1130203012,117675012,137560,77760,34060,9.604444
2,1020411930,110848063,143961,77580,36595,9.205501
3,954962863,105117642,141936,77580,34636,9.084706
4,911177709,101732938,134644,77580,34747,8.956565
5,980555941,108384820,138640,77580,36235,9.046986
6,846317735,95103854,144058,0,2724,8.898879
7,29551433,5179426,3593,0,2724,5.705542
